In [36]:
import numpy as np
import cvxpy as cp
import scipy.io
from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nn_weights_mat = scipy.io.loadmat('784-60-40-10weights.mat')

layer0 = nn_weights_mat['0.weight']  # 60 x 784
layer1 = nn_weights_mat['2.weight']  # 40 x 60
layer2 = nn_weights_mat['4.weight']  # 10 x 40
nn_weights = [layer0, layer1, layer2]

In [76]:
def relu(x):
    return np.clip(x, 0, None)


def forward(x, weight, bias):
    if bias:
        return relu(weight @ x + bias)
    else:
        return relu(weight @ x)
    

def ibp(x_tuple, weight, bias=None):
    x, x_ub, x_lb = x_tuple
    
    # run a normal forward pass
    z = forward(x, weight, bias)
    
    mu = (x_ub + x_lb) / 2
    r = (x_ub - x_lb) / 2
    mu = forward(mu, weight, bias)
    r = forward(r, np.abs(weight), bias)
    z_lb = mu - r
    z_ub = mu + r
    
    return z, z_ub, z_lb

In [77]:
n1, n0 = layer0.shape
random_x = np.random.random((n0, 1))
sigma_x = 1e-1 * np.ones((n0, 1))

In [78]:
x_tuple = (random_x, random_x + sigma_x, random_x - sigma_x)
z1, z1_ub, z1_lb = ibp(x_tuple, layer0)

In [79]:
# plt.figure(figsize=(15, 5))
# plt.plot(z1)
# plt.plot(z1_ub)
# plt.plot(z1_lb);

### MNIST
SDP for first layer. Input is 784 x 1.

- $D_{\eta}$: 784 x 784
- $D_{\lambda}$: 60 x 60
- $\mu$: 60 x 1
- $\nu$: 60 x 1
- $e_i$ or $\textbf{1}$: 60 x 1
- $W_0$: 60 x 784 

In [89]:
x0 = random_x.copy()
n1, n0 = layer0.shape

# create dual variables
eta = cp.Variable(n0)       # 784
D_eta = cp.diag(eta)        # 784 x 784
lambda_ = cp.Variable(n1)   # 60
D_lambda = cp.diag(lambda_) # 60 x 60
mu = cp.Variable((n1, 1))   # 60 x 1
nu = cp.Variable((n1, 1))   # 60 x 1
t = cp.Variable((1, 1))     # 1

# given variables
e = np.ones((n1, 1))        # 60 x 1
W0 = layer0.copy()          # 60 x 784

# formulate H matrix
c = eta @ (sigma_x ** 2 - x0 ** 2)
q = cp.vstack([e + mu + nu, -W0.T@nu + D_eta@x0])
Q = cp.vstack([cp.hstack([D_lambda, 0.5 * D_lambda@W0]), cp.hstack([0.5 * W0.T@D_lambda, -D_eta])])
H = cp.hstack([cp.vstack([t-c, q]), cp.vstack([q.T, -Q])])

constraints = [H >> 0, -Q >> 0, lambda_ >= 0, mu >= 0, nu >= 0, eta >= 0]
prob = cp.Problem(cp.Minimize(cp.trace(t)), constraints)
prob.solve(verbose=True)

# Print result.
print("The optimal value is", prob.value)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 145461, CG tol ~ 1/iter^(2.00)
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 965, constraints m = 714989
Cones:	linear vars: 964
	sd vars: 714025, sd blks: 2
Setup time: 4.62e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.22e+00  4.95e+02  9.99e-01 -5.47e+00  1.17e+03  0.00e+00  7.58e-01 
   100| 6.70e+18  3.26e+21  4.42e-01  2.01e+22  5.20e+22  3.16e+22  5.50e+01 
   200| 3.81e+18  1.89e+21  2.76e-01  2.17e+22  3.82e+22  1.

In [90]:
prob.value

66873.87370540718

In [91]:
z1_ub.max()

33.023116534551875